<font size="5"><center> <b>Sandpyper: sandy beaches SfM-UAV analysis tools</b></center></font>

    
<center><img src="images/banner.png" width="80%"  /></center>

<font face="Calibri">

# Multiscale sediment dynamics

<br>
<font size="3"> <b> Nicolas Pucino; PhD Student @ Deakin University, Australia </b> <br>

<b>This notebook covers the following concepts:</b>

- The ProfileDynamics class
- Multiscale volumetric analysis and plotting
- Hotspot analysis
- Multiscale beachface Cluster Dynamics indices and plotting
</font>

In [1]:
%matplotlib notebook

import matplotlib.pyplot as plt
import pickle
import pandas as pd
import seaborn as sb

from sandpyper.hotspot import ProfileDynamics
from sandpyper.dynamics import sensitivity_tr_rbcd, plot_sensitivity_rbcds_transects

pd.options.mode.chained_assignment = None  # default='warn'

C:\conda3\envs\sandpyper_env\lib\site-packages\pysal\explore\segregation\network\network.py:15: UserWarning: You need pandana and urbanaccess to work with segregation's network module
You can install them with  `pip install urbanaccess pandana` or `conda install -c udst pandana urbanaccess`
  warn(
C:\conda3\envs\sandpyper_env\lib\site-packages\pysal\model\spvcm\abstracts.py:10: UserWarning: The `dill` module is required to use the sqlite backend fully.
  from .sqlite import head_to_sql, start_sql
C:\conda3\envs\sandpyper_env\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
P=pickle.load(open(r"C:\my_packages\sandpyper\tests\test_data\test.p", "rb"))

SyntaxError: invalid syntax (profile.py, line 70)

___

## Introduction

In this notebook, we finally get some dynamics data out of our profiles using the __ProfileDynamics__ class.



### ProfileDynamics class

The __ProfileDynamics__ class is the core object in sandpyper. Its methods allows to:

* compute multitemporal elevation changes
* compute Limit of Detections (LoDs) statistics
* compute multi-scale volumetrics
* discretise elevation changes into discrete classes of magnite of changes with a variety of methods (thanks to the Python package [pysal](https://pysal.org/))
* compute multi-scale beachface cluster dynamics indices (BCDs, check the paper [Pucino et al., 2021](https://www.nature.com/articles/s41598-021-83477-6.epdf?sharing_token=844tVeK37MSYJoNEThYGItRgN0jAjWel9jnR3ZoTv0PxEZXr8bbQlsWb42PNMTsu53flRyY9FJ3fI5pe-Atcf1RbRfb5zpR7Q0KX8rhHKlN-viewjBsZvEcrYBcliJPElRIl7UYdYDwJS94n1E_4gLskgni0xLdXl8TujAjpfG8%3D))
* various plotting capabilities

In [ ]:
labels=["Undefined", "Small", "Medium", "High", "Extreme"]
appendix=["_deposition", "_erosion"]

Here below we instantiate a ProfileDynamics object which discretises the data into 5 bins using the Jenks and Caspall method. We also provides labels to the bins, in order to be able to name them and keep the analysis clear. 

In [ ]:
D = ProfileDynamics(P, bins=5, method="JenksCaspall", labels=labels)

### Computing multitemporal dataset (dh_df)

The method *ProfileDynamics.compute_multitemporal* allows to:

1. create pairwise (date from - date to) elevation difference dataset called dh_df
2. option to filter data based on a class. For instance, computing elevationc hanges of sand only
3. option to provide a dictionary to use full length location names instead of location codes in some plots (loc_full)

Once run, this will add an attribute called *dh_df* that stores the dataframe of elevation changes across all the surveys and time periods.
If LoD was previously derived with LoD transects, this method will also compute lod_dh table, which is used to create an LoD table.

Let's see how this work.

In [ ]:
D.compute_multitemporal(loc_full={'mar': 'Marengo',
         'leo': 'St. Leonards'}, filter_class='sand')

We can see that mutlitemporal statistics have been computed for each time period in all locations, creating the dh_df dataframe.

In [ ]:
D.dh_df.head()

The __dt__ column stores a 'time-period ID'. For instance, in St. Leonards, the changes of points from (20180606) to (20180713) is named dt_0, because it is the first time period in this location. The second one, (20180713) to (20180920) is named dt_1 and so on.

All the information about time periods are stored into a dataframe as an attribute called *ProfileDynamics.dh_details*.

In [ ]:
D.dh_details

In the same way the dh_df dataframes has been computed, the lod_dh table is also computed, except that no filters are applied and all points previously extracted from the LoD transects are used.

In [ ]:
D.lod_dh.head()

From this dataset, a set of useful error (i.e, elevation change in calibration areas where a theoretical zero change should occur) statistics is also computed, which include:

1. mean (column: 'mean')
2. median (column: 'med')
3. standard deviation (column: 'std')
4. normalised medain absolute deviation (column: 'nmad')
5. 68.3th quantile of absolute error (column: 'a_q683')
6. 95th quantile of absolute error (column: 'a_q95')
7. robust root mean squared error (column: 'rrmse')
8. number of total observations (column: 'n')
9. n_outliers using the 3-sigma method (column: 'n_outliers')
10. Shapiro–Wilk statistics, p-values and normality evaluation (columns: 'saphiro_stat', 'saphiro_p', 'saphiro_normality')
11. D’Agostino–Pearson statistics, p-values and normality evaluation (columns: 'ago_stat', 'ago_p', 'agoo_normality')
12. the chosen Limit of Detection based on normality check (column: 'lod')

All this information is stored in the **lod_df** attribute.

In [ ]:
D.lod_df

### Plot: LoD normality check

Eventhough statistical tests can be useful in identifying wether a distribution is normal or not, especially in the case of photogrammetrically derived products, it is advised to also visually check both the histograms and the Q-Q plots of absolute error.

Sandpyper allows quick and easy generation of such plots, here is an example for Marengo.

In [ ]:
D.plot_lod_normality_check(locations=['leo'])

## Multiscale volumetrics

With the LoD and dh_df tables we are finally ready to observe multi-scale change.

### Mean Elevation Change

The __Mean Elevation Change (MEC)__ is calculated with the totality of the valid points (filtered based on class, beyond LoD, within the beachface) that occur both in the pre and post dataset in each location.<br>
MEC is the preferred metric as it allows a robust comparison between locations and time, given a context where number of transects and retained valid points can drastically change from time to time or from location to locatio due to changes in:

- survey extent
- wave run-up and the swash level
- non-sand points across the beachface

### Volumetric estimation from profiles

The __volumes of beachface change__, instead, are estimated, using the following procedure.

1. Along each transect, the missing points (due to non-sand filters if applied) are replaced by a linearly interpolated value.
2. We integrate the change along the cleaned transect to estimated volumetric change at each transect.
3. We multiply the sum of all transect volume change by the transect spacing, to obtain a location-level beachface change estimation.

This approximation is necessary to go from transect change to beachface change. Here below is how Sandpyper takes care of that in one line, computing multi-scale MECs and volumetrics in one line, by just calling the *ProfileDynamics.compute_volumetrics* method.

This will add two additional dataframes as attributes, named __location_volumetrics__ and __transects_volumetrics__, which will store a number of statistics.

In [ ]:
# compute volumetrics

D.compute_volumetrics(lod=D.lod_df)

In [ ]:
D.location_volumetrics.head()

In [ ]:
D.transects_volumetrics.head()

Amongst the varios statistics computed (__check the docs for details__), the most important fields are:

- norm_net_change: abs_net_change/total number of points used in the calculation of this time period (n_obs_valid). This is the MEC.
- net_vol_change: Net volume change (m3)
- location_m3_m: Net volume change / beach length (m3/m)
- cum: Cumulative net volume change since the beginning of the monitoring (m3)
- cum_mec: Cumulative norm_net_change since the beginning of the monitoring (m3)

Let's use sandpyper to visualise these stats.

### Plot: multi-scale MECS and volumetrics

#### Transect-level

The method *ProfileDynamics.plot_transects* allow to visualise elevation profile changes in selected locations, tr_ids and time periods. COnveniently, if the data has been classified, by setting the parameter __classified=True__ we can also see what class each point was classified into.

>__NOTE__: if __classified=False__, the points that have been filtered out will not be displayed. If __classified=True__, then all points will be plotted. This is done to have a visual understanding of the cleaning procedure applied and appreciate what is actually retained to compute change statistics. 

In [ ]:
# let's plot transect ID 10 in Marengo, for dt_2 and dt_3 with no classification applied.

D.plot_transects(location='mar', tr_id=10, from_date='20180727', to_date='20180925', classified=False)

In [ ]:
# same, with classification applied.

D.plot_transects(location='mar', tr_id=10, dt=['dt_2','dt_3'], classified=True)

In the above example, we can see how a major sand nourishment interention built a considerable foredune during dt_2. However, as it is shown in dt_3, this intervation did't last long, as the sand got eroded and the profile started receding again.

To have a full overview of a transect history all in one place, Sandpyper can plot it for you.

In [ ]:
D.plot_transect_mecs(location='mar',tr_id=10)

In [ ]:
D.plot_transect_mecs(location='mar', lod=D.lod_df, tr_id=10)

Or, we can also have use Sandpyper to plot both alongshore heatmaps of altimetric change and alongshore estimated volume changes, in all locations and all time-periods.

In [ ]:
D.plot_alongshore_change(mode='subset',       # to plot only a subset of locations and dts, rather than all
                         dt_subset=['dt_2', 'dt_3'],
                         location_subset=["mar"],
                         lod=D.lod_df, # LoD will be applied when provided in form of lod dataframe or single (global) value
                         ax2_y_lims=(-10, 20), # lineplot y axis limits
                        fig_size=(10,6),
                        font_scale=.9,
                        plots_spacing=0, # spacing between the two plots. Try .1, it is also nice
                        bottom=True,
                        y_heat_bottom_limit=40, # how far seawar the heatmap will extent.
                        transect_spacing=D.ProfileSet.transects_spacing,
                        heat_yticklabels_freq=10,
                        heat_xticklabels_freq=10,)

#### Location-level

So far we saw data and plots at the transect level. Let's see a few plots at the location level, starting with the *ProfileDynamics.plot_single_loc* method.

In [ ]:
D.plot_single_loc(["mar"],None)

In the above plot we can appreciate the volume change timeseries of Marengo, with the solid line representing the period-specific change while the dashed line the cumulative change from the start of the monitoring.

Alternatively, we can plot both locations MECs timeseries in order to compare their elevational changes through time, using the method *ProfileDynamics.plot_mec_evolution*.

In [ ]:
# scale_mode can be 'auto', which means the x axis is location-specific, or 'equal', better for comparison.

D.plot_mec_evolution(location_field="location",x_limits=(-.2, .8), scale_mode='optimised',
                     loc_order=["leo","mar"],
                    x_binning=10,
                    figure_size=(6, 7),
                    font_scale=1.2)

In [ ]:
# scale_mode can be 'auto', which means the x axis is location-specific, or 'equal', better for comparison.

D.plot_mec_evolution(location_field="location",x_limits=(-.2, .8), scale_mode='equal',
                     loc_order=["leo","mar"],
                    x_binning=10,
                    figure_size=(6, 7),
                    font_scale=1.2)

Alternatively, if we want to change the x-axis of only specific locations, we can set the scale_mode='equal', provide x_lims, __and provide a dictionary where keys are the location codes and values the location specific limits.__ In this way we can keep all the analysis visually comparable, with some exceptions (different scale) provided with the dictionary in the __x_diff__ parameter.

```python
    x_diff = {'leo': [-0.12, 0.1]}
```

In [ ]:
x_diff={'leo':[-0.12, 0.1]}

D.plot_mec_evolution(location_field="location",x_limits=(-.2, .8), scale_mode='equal',
                     x_diff=x_diff,
                     dates_step=15,
                     loc_order=["leo","mar"],
                    x_binning=10,
                     sort_locations=False,
                    figure_size=(6, 7),
                    font_scale=1.2)

## Hotspot analysis: Local Indicator of Spatial Association (LISA)

In order to discard spatial outliers and get a better representation of the beachface dynamics by at the same time reducing the number of points analysed (in real-world case this can be an important aspect), we use the Local Moran's I index, which is one of the most used spatial autocorrelation measure, avalibale thanks to the great work in the Python package __[Pysal](https://pysal.org/)__.

The method __*ProfileDynamics.LISA_site_level*__ performs a Local Moran's I with False Discovery Rate (fdr) correction analysis for all the elevation change points in each survey in the __dh_df__ dataframe.

This method can use __KNN-based, inverse distance weighted or binary distance-based__ spatial weight matrices. In this example, we model spatial relationships with a distance-based row standardised binary weight matrix with neighborhood radius of __35 m__, in order to include two adjacent transect without getting too far from the focal point.

In [ ]:
D.LISA_site_level(mode="distance", distance_value=35)

This method adds some hotspot analysis relevant columns to the dh_df table, most importantly:

- false discovery rate threshold (column: lisa_fdr)
- local Moran's I (column: lisa_I)
- simulated pseudo p and z values (columns: lisa_p_sim, lisa_z_value)
- Moran's scatter plot quadrant (column: lisa_q)

__MORAN SCATTERPLOT__ (High-High cluster, High-Low spatial outlier, Low-Low cluster and Low-High spatial outlier)

Although technically the High-High and Low-Low Moran's scatter plot quadrants represent statistically significant spatial clusters, we commonly refer to these points as hotspots and coldspots.

>__IMPORTANT NOTE__: Hot versus cold spots do not necessary reflect positive and negative values, rather, they represent statistically significan changes in the magnitude of the process analysed (elevationc change). Thus, in a scenario of extremely high deposition, hotspots represents high values of deposition surrounded by other high values of deposition, while coldspots represent low values of deposition surrounded by other low values of deposition. The choice of Local Moran's I as LISA has been motivated by the fact that we want to represent dynamics as best as we can by removing spatial outliers at the location level and we are not necessarily interested (at this stage) at the actual human interpretation (deposition vs erosion).


Now that we have statistically significant hotspots of change, we can use those points to capture the most interesting areas of beachface change which we use to model behaviour.

>__ANOTHER NOTE__:
Our beachfaces are narrow, and we use only reliable valid points using multiple levels of filtering (LoD, beachface area, sand-only), which significantly reduce the total number of usable points in each timestep.
Because in our next step we will compute the Beachface Cluster Dynamics indices both at the location and transect scales, we will use the hotspot filtering only at the location level and not at the transect level.
This is necessary to assure that we have enough points in each transect to model their behaviours.

In [ ]:
D.hotspots.query("location=='mar' and dt=='dt_2' and lisa_q in [1,3]").plot(column='lisa_q', categorical=True,
                                                                            cmap='RdBu')

## Multiscale Beachface Cluster Dynamics (BCDs) indices

In this section, the Beachface Cluster Dynamics (BCDs), which compirse both the empirical and residual BCD indices, are computed with sandpyper.

> From [Pucino et al.](https://www.nature.com/articles/s41598-021-83477-6#Abs1),: " *The empirical and residual BCDs (e-BCD and r-BCD respectively) are purposefully designed metrics to leverage the very high spatiotemporal resolutions and three-dimensionality of our data for studying subaerial beach landform dynamics (morphodynamics). With elevation change (Δh) magnitude classes as transient states, we used finite discrete Markov chain models to compute first-order stochastic transition matrices and steady-state probability vectors, used to derive e- and r-BCD respectively.*" For more information about the methods, please refer to [the docs]().

As stated above, before computing any BCD, we need to discretise the magnitude of elevation change that occurred at every time step into bins.
When we intiate the ProfileSet instance we already provide sandpyper with all it need to perform the discretisation on the data.

Therefore it is as simple as calling the *ProfileDynamics.discretise* method, as shown below.

In [ ]:
D.discretise(absolute=True, print_summary=True, lod=D.lod_df)

By setting __absolute = True__, the bins will be derived from absolute values of elevation changes first, then, classes will be assigned to both negative and positive values accordingly. This is needed to ensure that both erosional and depositional classes represent the same interval in elevation change magnitudes, except of opposite sign.

After this step, a new dataframe is added named __df_labelled__, which stores the __markov_tag__ field, which is the result of the discretisation process.

In [ ]:
D.df_labelled.head()

Now, we are ready to compute the actual indices, but first, a note on the weights, neede for the e-BCDs.

### e-BCDs weights

The __e-BCD__ uses weights to represent the importance of each transition.<br>

A point that transitioned from small erosion to extreme erosion has more impact on the sediment budget than if it was transitioning to a medium erosion class.<br>
The weights for each magnitude class can be specified manually using a dictionary. However, it is best to obtain an objective representation of the severity of change. As the magnitude classes are derived from the data (using Jenks-Caspall optimised natural breaks method in this example), a simple yet robust solution is to use the medians of each magnitude class as weight.
<br>
<br>
The function __infer_weights__ takes care of this process.

In [ ]:
D.infer_weights()

D.weights_dict

Of course, if we want to use our own designed weigths, it is as simple as this below:

In [ ]:
# create the weight dictionary

my_weights_dict={'Undefined_deposition': 1,
 'Undefined_erosion': 1,
 'Small_deposition': 1,
 'Small_erosion': 1,
 'Medium_deposition': 1,
 'Medium_erosion': 1,
 'High_deposition': 3,
 'High_erosion': 3,
 'Extreme_deposition': 8,
 'Extreme_erosion': 8}

# add it to the ProfileDynamics object

D.weights_dict = my_weights_dict

D.weights_dict

However, it makes more sense to use Sandpyper in-built method, so, let's recalculate it.


In [ ]:
D.infer_weights()

D.weights_dict

Now, we can compute BCDs at the location level at once.

The method *ProfileDynamics.BCDs_compute* computes all the stochastic first-order transition matrices of sand dynamics, based on the filters provided (sand-only, hotspots, beyond LoD) across beachfaces, at the site level, which are the basis for the e-BCDs.

It returns 2 dataframes:
* __e-BCDs__
* __steady-state distribution__

In [ ]:
D.BCD_compute_location("geometry","all",True, filterit='lod')

After this step, two new dataframes are added as attributes to the ProfileDynamics object:
1. location_ebcds : stores the location level e-BCDs of each submatrix, as well as its value, sign and trend.
2. location_ss : stores the location level steady-state vector of each magnitude of change class

In [ ]:
D.location_ebcds

In [ ]:
D.location_ss


The residual is simply the difference between erosional and depositional probabilities in the Steady-State distribution, __multiplicated by 100__ for readability purposes. Note that in this case, no weigths are applied as no transition is represented.


Here below, the dataframe returns the residual column, which is what you might want to map in Qgis.

The above table, the row __r_bcds__ represent the r-BCD index for the locations, which we also call __behavioural regime__.

### Plot: first-order transition stochastic matrices

Let's use sandpyper to visualise location-specific first-order stochastic matrices, representing the probability ofeach transiction, which is the basis for the computation of the e-BCDs.

In [ ]:
# the following dictionary is used to rename the magnitude of change classes into small codes
#such as 'me' for Medium Erosion or 'sd' for Small Deposition. This is done purely for graphical purposes.

relabel_dict={'Undefined_deposition': 'ue',
 'Undefined_erosion': 'ue',
 'Small_deposition': 'sd',
 'Small_erosion': 'se',
 'Medium_deposition': 'md',
 'Medium_erosion': 'me',
 'High_deposition': 'hd',
 'High_erosion': 'he',
 'Extreme_deposition': 'ed',
 'Extreme_erosion': 'ee'}

In [ ]:
%matplotlib inline

#let's plot those matrices

D.plot_trans_matrices(relabel_dict)

Notes: <br>
>These matrices discard all the __valid to non-valid transitions__. In other words, all transitions going from a valid point (non spatial outlier, classified as sand and beyond limit of detection) are labelled as "nnn" and discarded.
Moreover, __the colour ramp__ higher limit (vmax parameter) is set to a maximum of 3 times the standard deviation of all dataset without the nnn state.

#### How to read transition matrices

__The title__ informs about:
* mar/leo = Location codes (leo = St. Leonards; mar = Marengo)
* n = Total number of points in the timeseries (valid and non-valid)
* t = Total number of timesteps
* trans = Total number of valid transitions considered

__Rows__ are the initial state, while the __columns__ are the final (one-step) state. For instance, in Marengo, a point which starting state is Undefined Erosion (ue) has 12% probability to become a Medium Erosion (me) point and a 15% probability to become a Medium Deposition in the next time period.  

__Some observations__
>We chose St. Leonards as an example to demonstrates a few limitations, which are observable in this matrix.

__St. Leonards__ is located within Port Phillip bay. This narrrow beach is not embayed, but __fetch-limited__, meaning that it doesn't receive the highly energetic Souther Ocean swell, rather, it sees its morphodynamics impacted by considerably lower waves which are locally generated from within the bay. Moreover, seagrass meadows and reefs further reduce wave imapcts on the subaerial beachface.<br>
Therefore, __changes are small__ and often below the limit of detections, thus, uncertain. If we consider that we filter points based on:

1. only sand
2. limit of detection
3. spatial outliers

from an already narrow beach (short transects) this location is severly decimated in terms of behavioural modelling potential. In fact, the submatrices have many zeroes, which indicate a zero probability of these transitions, because these have not been recorded during the monitoring period.
Moreover, we note how most of the transitions tend to be __from lower magnitude classes to low magnitude classes.__
Another story for Marengo. <br>

__Marengo__ is an open-ocean beach with a small southern section pretected by a small headland (tombolo-like). The section used for this example is also affected by sand nourishment, which injects considerable amount of sediment during the monitoring period.<br>
Given in Marengo we have roughly 4 times the number of valid observations (trans=5'278) as compared to St. Leonards (trans=1'072), the submatrices better capture the stochastic behaviour of this beachface.



### Plot: e-BCDs (location)

Let's plot the e-BCDs, which summarises the information contained in the submatrices.

In [ ]:
D.plot_location_ebcds()

### Plot: r-BCD (location)

We already computed the __Steady State__ probability vectors for each location using the __BCDs_compute__ function, storing it in the __ss__ variable.<br>
Here below we reorder the data and plot as an heatmap, where each columns is a location and each row a magnitude of change.


In [ ]:
order=[i for i in D.tags_order if i !='nnn']
ss=D.location_ss.loc[order] 
ss

In [ ]:
plt.rcParams['font.sans-serif'] = 'Arial'
plt.rcParams['font.family'] = 'sans-serif'
sb.set_context("paper", font_scale=2)


f,ax=plt.subplots(figsize=(12,10))

sb.heatmap(ss, cmap="Blues",annot=True,
           annot_kws={'size':14},linewidths=1,linecolor="white", cbar_kws={'label': 'Lim. Probabilities'});

#f.savefig(r'E:\\path\\to\\save\\picture.png', dpi=600); 

Location level BCDs are good to compare multiple locations across wide areas. But what about getting a more detailed spatial-explicit view of the behaviour of a beachface system, at the __transect-level__?

### Transect-level BCDs 

If we are looking at spatial distribution of r-BCDs, it is important that transects are comparable to each other within one location. 

So far, LoDs and sand-only filters have assured that only high quality data was used in the r-BCDs computation. At the transect level we add two additional conditions, to ensure that are only retained:

1. points that remain valid for at least a certain amount of time periods
2. transects that have a number of valid points greater than a determined minimum threshold

Therefore, the choice of the two parameters (__thresh and min_pts__) of the *ProfileDynamics.BCD_compute_transects* method is very important:

1. __min_points__: the minimum required valid points per transect to consider a transect reliable.
2. __thresh__: the minimum number of timesteps required to retain a transect.

This is done to ensure comparability across time and transects in a determined location.
Therefore, __thresh__ is an important parameter that should ideally be as high as the available time periods, in order to ensure that the maximum behavioural variability is captured.<br>
Yet, setting this parameter very high can reduce considerably the number of valid points retained in a single transect, which in turn can fall below __min_pt__, leading to the loss of transects from the final behavioural map.<br>


Let's see how this works.

In [ ]:
loc_specs={'mar':{'thresh':6,
       'min_points':6}}

D.BCD_compute_transects(loc_specs=loc_specs,reliable_action='keep', dirNameTrans=D.ProfileSet.dirNameTrans)

In [ ]:
D.transects_rbcd.query("location=='leo'").plot(column="residual", cmap='RdBu_r')

Done. We now have transect level r-BCDS computed with the totality of elevation changes transitions occurred during the monitoring period.


#### Sensitivity analysis

However, as previously discussed, the choice of the __thresh and min_pts__ parameter is very important. It is also informative to monitor and try to minimise the number of transects that __passed from a depositional to erosional behavioral regime (or vice versa) in the last time period (i. e. changed sign from t-1 to t),__ for any chosen __thresh__. Those transects could signal a behaviour that only emerged by choosing a determined value for __thresh__, signaling a potentially __lower confidence in their r-BCD values__.<br>Also  choosing the right min_pts and thresh parameters is important.

This is why sandpyper includes 2 functions to perform a __sensitivity analysis__:

1. __sensitivity_tr_rbcd()__ : to perform a sensitivity analysis 
2. __plot_sensitivity_rbcds_transects()__ : to visualise the sensitivity analysis results


We start by defining all the combinations between the parameters __thresh (t)__ and __min_pts (pt)__, given ranges that we define.<br>
The thresh parameter can be up to the total number of timesteps available, while the min_pts, we decide to test the values from 0 to 210, with a step of 10.

In [ ]:
ss_tr_big = sensitivity_tr_rbcd(D.df_labelled,
                                test_thresholds='max',
                                test_min_pts=[0,20,2])

In [ ]:
f,ax=plt.subplots(figsize=(10,10))

palette=sb.color_palette( n_colors=ss_tr_big.tr_id.unique().shape[0])
sb.lineplot(data=ss_tr_big, x='thresh',y='residual', hue='tr_id', style='location',
            palette=palette, legend=False, **dict(alpha=0.5),
            ax=ax
)
ax.set_ylabel("r_bcd")
ax.axhline(y=0, lw=2, c='r');

As we can see, there are only 3 transects that flipped their r-BCD sign (depo to ero or viceversa) as a consequence of an increase in the minimum number of timesteps.

In order to better decide a sub-optimal combination of thresh and min_pts that retains the majority of the transects (while keeping an eye on transects that flipped r-BCD), here below we do some plotting.

In [ ]:
plot_sensitivity_rbcds_transects(ss_tr_big, location='mar')

## Conclusion

This notebook concludes the tutorials for the profile-based monitoring approach offered by sandpyper. In the next notebook we introduce a paradigm shift, which is raster-based analysis. However, this part of sandpyper is currently under development, thus, only a small __teaser notebook__ will illustrate a few useful functions already implemented in sandpyper.

See you there!